In [4]:
# 모델 정의

import os
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import *
import torch
import math
from os.path import join
import torch.utils.data as data
import numpy as np
from os import listdir
from os.path import join
from PIL import Image, ImageOps
import random
from random import randrange
from math import sqrt
from torchsummary import summary

'''
1. Channel_weighted_Conv
   어떤 '한개의' filter 에 대해, 
   입력 [r^2, N, N] 인 feature map 의 각 channel 의 conv. 결과를 순차적으로 mapping 한 [1, rN, rN] 크기의 feature map 생성하고,
   생성한 feature map 에 rxr inner filtering 을 적용한 [1, N, N] 크기의 결과를 출력하는 함수 
'''
class Channel_weighted_Conv(nn.Module):
  def __init__(self, input_size, kernel_size, stride=1, bias=True):
    super(Channel_weighted_Conv, self).__init__()
    self.input_size = input_size 
    # input_size = r^2

    self.conv = torch.nn.Conv2d(1, 1, kernel_size, stride, padding=int((kernel_size-1)/2), bias=bias)
    # 입력 feature map [batch, r^2, N, N]에서 [batch, i, N, N] 을 입력으로 받아 1 개의 feature map 출력
    # 출력이 [batch, i, N, N] 가 되게 하기 위해 padding = int((kernel_size-1)/2)
    self.PS =  nn.PixelShuffle (upscale_factor = int(math.sqrt(input_size)))
    # upscale_factor = r
    self.inner_conv = torch.nn.Conv2d(1, 1, kernel_size=int(math.sqrt(input_size)), stride=int(math.sqrt(input_size)), padding=0, bias=bias)

    '''
    apply 함수로 initialization 하기 힘들기 때문에 class 내에서 initialization 을 수행하도록 함
    '''
    for m in self.modules():
        # 이 class 에서 정의한 self.(변수)[인스턴스 변수] 목록 (conv, PS, inner_conv) 불러오기 
            classname = m.__class__.__name__
            # 불러온 module의 class 이름
            if classname.find('Conv2d') != -1:
                torch.nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    m.bias.data.zero_()
            # Conv2d Layer 라면 He 방법으로 해당 Layer 의 weight 들을 초기화, bias 가 True 면 bias 를 0 으로 초기화
            elif classname.find('ConvTranspose2d') != -1:
                torch.nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    m.bias.data.zero_()
           # ConvTranspose2d Layer 라면 He 방법으로 해당 Layer 의 weight 들을 초기화, bias 가 True 면 bias 를 0 으로 초기화

  def forward(self, x):
    s = self.conv(x[:, 0:1, :, :])

    for i in range(1, self.input_size):
      m = self.conv(x[:, i:i+1, :, :])
      s = torch.cat([s, m], dim=1)
      
    # s.shape = [batch, r^2, N, N]
    # s = 입력 feature map의 각 channel에 대해 한개의 filter 와의 단순 convolution 결과 feature map
    
    ps = self.PS(s)
    # ps.shape = [batch, 1, rN, rN]
    # ps = 한 filter에 대해 입력 feature map 의 각 channel 가중치를 학습하기 전 feature map

    out = self.inner_conv(ps)
    # out.shape = [batch, 1, N, N]
    # out = 가중치를 학습하고 난 후 필터링 결과
    return out

'''
2. WCNN_block 
   기존의 Conv2d 와 같은 입력을 받아, 각 Channel 의 가중치 (inner_filter) 를 학습하는 과정을 추가하여
   입력과 동일한 크기의 output_size 개 만큼의 feature map 을 출력하는 New Conv2d layer
'''
class WCNN_block(nn.Module):
  def __init__(self, input_size, output_size, kernel_size):
    super(WCNN_block, self).__init__()
    self.filters = [0]*output_size

    '''
    여기서 filter 갯수(=output_size)만큼 Channel_weighted_Conv 을 list 형태로 담고,
    '''
    self.filters = nn.ModuleList([Channel_weighted_Conv(input_size, kernel_size, stride=1, bias=True) for _ in range(0, output_size)])
    self.filter_num = output_size
  
  def forward(self, x):
    '''
    여기서 각 Channel_weighted_Conv에 x(입력) 를 대입하고 나온 출력을 concatenation 해 준다.
    -> ModuleList 는 list 안의 각 Module 을 하나씩 접근 할 수 있게 해 줌
    '''
    out = self.filters[0](x)
    for i in range(1, self.filter_num):
      filtering = self.filters[i](x)
      out = torch.cat([out, filtering], dim=1)
    # out.shape = [batch, K, N, N]
    # out = 각 filter 들의 WCNN 결과의 concatenation
    return out

'''
3. ConvBlock
'''
class ConvBlock(nn.Module):
    def __init__(self, input_size, output_size, kernel_size=3, stride=1, padding=1, bias=True, activation='relu', norm=None):
        super(ConvBlock, self).__init__()
        self.conv = torch.nn.Conv2d(input_size, output_size, kernel_size, stride, padding, bias=bias)

        self.norm = norm
        if self.norm =='batch':
            self.bn = torch.nn.BatchNorm2d(output_size)
        elif self.norm == 'instance':
            self.bn = torch.nn.InstanceNorm2d(output_size)
        
        # self.bn : Conv Layer 출력에서 normalization 을 Instance 로 할지 Batch 로 할지 선택
        
        self.activation = activation
        if self.activation == 'relu':
            self.act = torch.nn.ReLU(True)
        elif self.activation == 'prelu':
            self.act = torch.nn.PReLU()
        elif self.activation == 'lrelu':
            self.act = torch.nn.LeakyReLU(0.2, True)
        elif self.activation == 'tanh':
            self.act = torch.nn.Tanh()
        elif self.activation == 'sigmoid':
            self.act = torch.nn.Sigmoid()
        
        # self.act : Conv Layer 출력 Activation Function 선택

        for m in self.modules():
            classname = m.__class__.__name__
            if classname.find('Conv2d') != -1:
                torch.nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    m.bias.data.zero_()
            # Conv2d Layer 라면 He 방법으로 해당 Layer 의 weight 들을 초기화, bias 가 True 면 bias 를 0 으로 초기화
            elif classname.find('ConvTranspose2d') != -1:
                torch.nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    m.bias.data.zero_()
           # ConvTranspose2d Layer 라면 He 방법으로 해당 Layer 의 weight 들을 초기화, bias 가 True 면 bias 를 0 으로 초기화
        
    def forward(self, x):
        if self.norm is not None:
            out = self.bn(self.conv(x))
        else:
            out = self.conv(x)

        if self.activation is not None:
            return self.act(out)
        else:
            return out

'''
4. WCNN
'''
class WCNN(nn.Module):
    def __init__(self):
        super(WCNN, self).__init__()
        
        '''
        1. layer2 만을 WCNN_block 으로 바꿈
        2. WCNN_block 의 input_size = r^2 이어야 하는 것을 유의함
        '''
        self.layer1 = ConvBlock(3, 64, kernel_size=9, stride=1, padding=4, activation='relu', norm=None)
        self.layer2 = nn.Sequential(
            WCNN_block(input_size=64, output_size=32, kernel_size=3),
            nn.ReLU(True),
        )
        # WCNN_block 의 Kernel_size = 홀수 여야 출력이미지의 size 가 입력과 같아진다
        self.layer3 = self.layer3 = ConvBlock(32, 3, kernel_size=5, stride=1, padding=2, activation=None, norm=None)

    def forward(self, x):
        f1 = self.layer1(x)
        f2 = self.layer2(f1)
        y = self.layer3(f2)
        
        return y
print('finish')


finish


In [ ]:
# 모델 확인

import os
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from google.colab import drive


drive.mount('/content/gdrive')

'''
1. GPU 설정 및 parameter print
'''
gpus_list = range(1)
cudnn.benchmark = True

cuda = True
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

model = WCNN()

'''
4. GPU 사용 여부 및 pretrained model 사용 여부 설정
'''
if cuda:
    model = model.cuda(gpus_list[0])
    model = torch.nn.DataParallel(model, device_ids=gpus_list)
    
summary(model, (3,32,32))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]          15,616
              ReLU-2           [-1, 64, 32, 32]               0
         ConvBlock-3           [-1, 64, 32, 32]               0
            Conv2d-4            [-1, 1, 32, 32]              10
            Conv2d-5            [-1, 1, 32, 32]              10
            Conv2d-6            [-1, 1, 32, 32]              10
            Conv2d-7            [-1, 1, 32, 32]              10
            Conv2d-8            [-1, 1, 32, 32]              10
            Conv2d-9            [-1, 1, 32, 32]              10
           Conv2d-10            [-1, 1, 32, 32]              10
           Conv2d-11            [-1, 1, 32, 32]              10
           Conv2d-12            [-1,

In [5]:
# 데이터 전처리 & 사용자 정의 data

from __future__ import print_function
import argparse

import os
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import *
import torch
import math
from os.path import join
import torch.utils.data as data
import numpy as np
from os import listdir
from os.path import join
from PIL import Image, ImageOps, ImageFilter
import random
from random import randrange
from math import sqrt

'''
1. filename 에 .png, .jpg, .jpeg 중 하나라도 있으면 True 를 반환하는 함수
'''
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".png", ".jpg", ".jpeg"])

'''
2. PIL 을 이용하여 filepath에서 이미지를 불러오고 RGB 로 convert 하여 반환하는 함수
'''
def load_img(filepath):
    img = Image.open(filepath).convert('RGB')
    return img

'''
3. PIL 을 이용하여 이미지를 Bicubic 으로 scale 만큼 High Resolution 으로 만드는 함수
'''
def rescale_img(img_in, scale):
    size_in = img_in.size
    # size_in = img_in 의 (width, height) tuple
    new_size_in = tuple([int(x * scale) for x in size_in])
    # new_size_in = (width * scale, height * scale)
    img_in = img_in.resize(new_size_in, resample=Image.BICUBIC)
    return img_in

'''
4. img_tar = load_img 함수를 이용하여 index 번째 image 를 RGB 로 불러온 이미지 
   img_in = target 으로 불러온 이미지를 Bicubic 으로 downscale 한 이미지 -> (사용할 일 없음)
   img_bic = rescale_img 함수로 input 을 다시 같은 비율로 upscale 한 이미지

   training 에는 img_tar, img_bic 을 사용할 것이고, 이들 이미지 임의의 영역을
   upscale_factor*n x upscale_factor*n 크기로 잘라 이미지 크기를 같게 만들어 줌
   
   잘린 이미지는 같은 random point 에서 시작하여 잘리므로 이미지가 나타내는 임의의 영역이 같음
'''
def get_patch(img_in, img_tar, img_bic, patch_size, scale, ix=-1, iy=-1):
    (ih, iw) = img_in.size
    (th, tw) = (scale * ih, scale * iw)

    patch_mult = scale #if len(scale) > 1 else 1
    tp = patch_mult * patch_size
    # tp = upscale_factor*n (n = patch_size)
    ip = tp // scale
    # ip = n (n = patch_size)
    
    if ix == -1:
        ix = random.randrange(0, iw - ip + 1)
    if iy == -1:
        iy = random.randrange(0, ih - ip + 1)

    (tx, ty) = (scale * ix, scale * iy)

    img_in = img_in.crop((iy,ix,iy + ip, ix + ip))
    # 원본이미지 임의의 영역을 n x n 크기로 자름
    img_tar = img_tar.crop((ty,tx,ty + tp, tx + tp))
    # downscale 한 이미지 임의의 영역을 upscale_factor*n x upscale_factor*n 크기로 자름
    img_bic = img_bic.crop((ty,tx,ty + tp, tx + tp))
    # upscale 한 이미지 임의의 영역을 upscale_factor*n x upscale_factor*n 크기로 자름            
    info_patch = {
        'ix': ix, 'iy': iy, 'ip': ip, 'tx': tx, 'ty': ty, 'tp': tp}
    # info_patch 에는 어떤 크기로 crop 했는지에 대한 정보가 있음
    return img_in, img_tar, img_bic, info_patch

'''
5. 이미지데이터 증강(augmentation)
'''
def augment(img_in, img_tar, img_bic, flip_h=True, rot=True):
    info_aug = {'flip_h': False, 'flip_v': False, 'trans': False}
    
    if random.random() < 0.5 and flip_h:
        img_in = ImageOps.flip(img_in)
        img_tar = ImageOps.flip(img_tar)
        img_bic = ImageOps.flip(img_bic)
        info_aug['flip_h'] = True
    # 50%의 확률로, flip_h = True 면 img_in, img_tar, img_bic 을 모두 flip

    if rot:
        if random.random() < 0.5:
            img_in = ImageOps.mirror(img_in)
            img_tar = ImageOps.mirror(img_tar)
            img_bic = ImageOps.mirror(img_bic)
            info_aug['flip_v'] = True
        if random.random() < 0.5:
            img_in = img_in.rotate(180)
            img_tar = img_tar.rotate(180)
            img_bic = img_bic.rotate(180)
            info_aug['trans'] = True
    # 50%의 확률로, rot = True 면 img_in, img_tar, img_bic 을 모두 mirror & 180도 rotate
    
    return img_in, img_tar, img_bic, info_aug
    # info_aug = 출력된 이미지에 어떤 preprocessing 을 했는지 알려줌
    
'''
6. PIL 이미지나 ndarray 이미지를 Tensor로 바꾸어주는 함수
   -> ToTensor 함수 사용시 이미지 pixel 값은 0~255 에서 0~1 로 바뀜
'''
def transform():
    return Compose([ToTensor(),])

'''
7.사용자 정의 data 로드 - Training Data 
'''        
class DatasetFromFolder(data.Dataset):
    '''
    def __init__(self, index) 에서는 필요한 변수를 선언 하고 'data 경로' 를 load 한다
    '''
    def __init__(self, image_dir, patch_size, upscale_factor, data_augmentation, transform=None):
        super(DatasetFromFolder, self).__init__()
        self.image_filenames = [join(image_dir, x) for x in listdir(image_dir) if is_image_file(x)]
        # self.image_filenames = image_dir 에 있는 모든 이미지의 경로 list
        self.patch_size = patch_size
        self.upscale_factor = upscale_factor
        self.transform = transform
        self.data_augmentation = data_augmentation
    '''
    def __getitem__(self, index) 에서는 index 번째 data를 return 하도록 코드를 짠다
    '''
    def __getitem__(self, index):
        target = load_img(self.image_filenames[index])
        # target = load_img 함수를 이용하여 index 번째 image 를 RGB 로 불러온 이미지
        input = target.resize((int(target.size[0]/self.upscale_factor),int(target.size[1]/self.upscale_factor)), Image.BICUBIC) 
        # input = target 으로 불러온 이미지를 Bicubic 으로 downscale 한 이미지
        bicubic = rescale_img(input, self.upscale_factor)
        # bicubic = rescale_img 함수로 input 을 다시 같은 비율로 upscale 한 이미지
        
        input, target, bicubic, _ = get_patch(input,target,bicubic,self.patch_size, self.upscale_factor)
        # input, target, bicubic 이미지를 (upscale_factor*patch_size x upscale_factor*patch_size) 로 crop
        
        if self.data_augmentation:
            input, target, bicubic, _ = augment(input, target, bicubic)
        # data_augmentation = True 라면, 세 이미지를 augment 함수로 증강, img_aug 는 필요 없음
        
        if self.transform:
            input = self.transform(input)
            bicubic = self.transform(bicubic)
            target = self.transform(target)
        # self.transform = True 라면, 세 이미지를 PIL image 에서 Tensor로 바꾸어 줌
        return input, target, bicubic
    '''
    def __len__(self) 에서는 data의 len을 return 하도록 코드를 짠다
    '''
    def __len__(self):
        return len(self.image_filenames)
    
'''
8.사용자 정의 data 로드 - Test Data 
   -> file 이름까지 사용자 정의 데이터화 시킴
   -> get_patch 함수를 사용하지 않기 때문에, SRCNN 의 입력으로 (원본 이미지 xscale) 된 이미지가 들어감
'''
class DatasetFromFolderEval(data.Dataset):
    def __init__(self, lr_dir, upscale_factor, transform=None):
        super(DatasetFromFolderEval, self).__init__()
        self.image_filenames = [join(lr_dir, x) for x in listdir(lr_dir) if is_image_file(x)]
        self.upscale_factor = upscale_factor
        self.transform = transform

    def __getitem__(self, index):
        target = load_img(self.image_filenames[index])
        # target = load_img 함수를 이용하여 index 번째 image 를 RGB 로 불러온 이미지
        _, file = os.path.split(self.image_filenames[index])
        # file = 이미지 경로 list 에서 file 이름만 잘라 냄

        input = target.resize((int(target.size[0]/self.upscale_factor),int(target.size[1]/self.upscale_factor)), Image.BICUBIC) 
        # input = target 으로 불러온 이미지를 Bicubic 으로 downscale 한 이미지
        bicubic = rescale_img(input, self.upscale_factor)
        # bicubic = rescale_img 함수로 input 을 다시 같은 비율로 upscale 한 이미지
        
        if self.transform:
            input = self.transform(input)
            bicubic = self.transform(bicubic)
            target = self.transform(target)
         # self.transform = True 라면, 세 이미지를 PIL image 에서 Tensor로 바꾸어 줌
        
        return input, bicubic, target, file
        # file 이름까지 return
      
    def __len__(self):
        return len(self.image_filenames)
'''
9. Training Data 불러오기
'''
def get_training_set(data_dir, hr, upscale_factor, patch_size, data_augmentation):
    hr_dir = join(data_dir, hr)
    return DatasetFromFolder(hr_dir,patch_size, upscale_factor, data_augmentation,
                             transform=transform())
'''
10. Test Data 불러오기
'''
def get_eval_set(lr_dir, upscale_factor):
    return DatasetFromFolderEval(lr_dir, upscale_factor,
                             transform=transform())

print('finish')


finish


In [ ]:
# Data Load & Model Load & Optimizer 
# Training

from __future__ import print_function
import argparse
from math import log10

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import pdb
import socket
import time
import easydict

from torch.utils.tensorboard import SummaryWriter

from google.colab import drive
drive.mount('/content/gdrive')

opt = easydict.EasyDict({ 
    "batchSize": 32,           # batch size - 한번에 training할 patch의 숫자
    "lr": 0.001,                # learning rate
    "upscale_factor": 2,       # Upscale 정도
    "patch_size": 64,         # 입력 이미지 크기

    "start_epoch": 1,           
    "nEpochs": 100,            # epoch 횟수
    "snapshots": 10,           # weight 저장 주기
   
    "data_dir":"/content/gdrive/MyDrive/졸업논문/data_800개/train", # dataset이 저장된 위치
    "hr_train_dataset": "DIV2K_train_HR", # training에 사용할 dataset 종류

    "model_type": "WCNN",         # 모델이름
    "save_folder": "/content/gdrive/MyDrive/졸업논문/졸업논문/scale=2/insize=3_lr=0.001_patch=64/weights/", # weight 저장 위치

    "pretrained_sr": None, # pretrained model 경로
    "pretrained": False,
    "data_augmentation": False,
    "gpu_mode": True,
    "threads": 1, 
    "gpus": 1 # 사용할 gpu 번호
    })

'''
1. Training Data 로드
'''
print('===> Loading datasets')
train_set = get_training_set(opt.data_dir, opt.hr_train_dataset, opt.upscale_factor, opt.patch_size, opt.data_augmentation)
training_data_loader = DataLoader(dataset=train_set, num_workers=opt.threads, batch_size=opt.batchSize, shuffle=True)

'''
2. WCNN 빌드 및 가중치 초기화 (가중치 초기화는 빌드와 동시에 수행)
'''
print('===> Building model ', opt.model_type)
model = WCNN()
print('===> Weight Initialization ')

'''
3. Optimizer & Criterion 정의 
  -> MSE loss 사용
  -> Optimizer는 Adam 사용
'''
criterion = nn.MSELoss() 
optimizer = optimizer = optim.Adam(model.parameters(), lr=opt.lr, betas=(0.9, 0.999), eps=1e-8)
# betas = SGD + Momentum 에서 momentum값, 0.9에서 시작하여 0.999로 증가

'''
4. GPU 설정 및 parameter print
'''
gpus_list = range(opt.gpus)
cudnn.benchmark = True
print(opt)

cuda = opt.gpu_mode
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

'''
5. GPU 사용 여부 및 pretrained model 설정
'''
if cuda:
    model = model.cuda(gpus_list[0])
    model = torch.nn.DataParallel(model, device_ids=gpus_list)
    criterion = criterion.cuda(gpus_list[0])
    
if opt.pretrained:
    model_name = os.path.join(opt.pretrained_sr)
    checkpoint = torch.load(model_name)

    model.load_state_dict(checkpoint['model_state_dict'])
    loss = checkpoint['loss']
    opt.start_epoch = checkpoint['epoch']
    print('Pre-trained SR model is loaded.')
    # pretrained model 불러오기

'''
6. 한번의 epoch 에서 수행하는 과정 정의 및 Tensorboard
'''
%load_ext tensorboard

%tensorboard tensorboard --logdir=/content/gdrive/My Drive/졸업논문/졸업논문/scale=2/logs_scale=2/ --port=6006

writer = SummaryWriter('/content/gdrive/My Drive/졸업논문/졸업논문/scale=2/logs_scale=2/')
# log 를 저장할 위치 지정

loss_list = []

def train(epoch):
    epoch_loss = 0
    model.train()
    # model.train() 을 호출하여 학습 모드로 전환
    '''
    이때, 각 이미지는 'mini batch' 갯수만큼 가져온다!
    이때, autograd.Variable 을 사용하는 이유는 batch_data 라는 list 에 있는 input, target, blur_img 를 Tensor로 불러오기 위해
    '''
    for batch_idx, batch_data in enumerate(training_data_loader, start=1):
    # batch_data = (input[batch_idx], target[batch_idx], blur_img[batch_idx])
    # batch_idx = 세개의 mini-batch의 출력 이미지 set의 index
    # enumerate({list}, number) 로 사용하면 interation 이 0 이 아니라 number 부터 시작 함
        _, target, bicubic = Variable(batch_data[0]), Variable(batch_data[1]), Variable(batch_data[2])
        # autograd.Variable({tensor}) 라고 쓰면, {tensor} 를 불러옴 
        if cuda:
            # input = input.cuda(gpus_list[0])
            target = target.cuda(gpus_list[0])
            bicubic = bicubic.cuda(gpus_list[0])

        optimizer.zero_grad()
        # gradient 를 0으로 초기화
        t0 = time.time()
        prediction = model(bicubic)

        loss = criterion(prediction, target)
        epoch_loss += loss.data
        # 해당 mini-batch 에서 loss function 값
        loss.backward()
        # loss 를 가지고 gradient 계산
        optimizer.step()
        # 각 layer들의 weights 갱신
        t1 = time.time()

        print("===> Epoch[{}]({}/{}): Loss: {:.4f} || Timer: {:.4f} sec.".format(epoch, batch_idx, len(training_data_loader), loss.data, (t1 - t0)))
        # t1-t0 = 한 mini-batch 연산에 걸린 시간

    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(training_data_loader)))
    
    '''
    Model 의 Checkpoint 저장하기 for test(evaluation)
    '''
    if (epoch) % (opt.snapshots) == 0:
      model_out_path = opt.save_folder+"WCNN_epoch_{}.pth".format(epoch)
      torch.save({'epoch' : epoch,
              'model_state_dict' : model.state_dict(),
                'loss' : loss.data}, model_out_path)
      print("Checkpoint saved to {}".format(model_out_path))

    '''
    Tensorboard 에 매 epoch 마다 loss 저장하기
    '''
    writer.add_scalar('Epoch Loss', epoch_loss / len(training_data_loader), epoch )
    # 그래프의 가로축 = epoch, 세로 축 = epoch_loss / len(training_data_loader)

    loss_list.append(epoch_loss / len(training_data_loader))

if __name__ == '__main__':
    for epoch in range(opt.start_epoch, opt.nEpochs + 1):
        train(epoch)

writer.close()
# writer 가 더이상 필요하지 않으므로 닫아준다

print('Finished Training')


Mounted at /content/gdrive
===> Loading datasets
===> Building model  WCNN
===> Weight Initialization 
{'batchSize': 32, 'lr': 0.001, 'upscale_factor': 2, 'patch_size': 64, 'start_epoch': 1, 'nEpochs': 100, 'snapshots': 10, 'data_dir': '/content/gdrive/MyDrive/졸업논문/data_800개/train', 'hr_train_dataset': 'DIV2K_train_HR', 'model_type': 'WCNN', 'save_folder': '/content/gdrive/MyDrive/졸업논문/졸업논문/scale=2/insize=3_lr=0.001_patch=64/weights/', 'pretrained_sr': None, 'pretrained': False, 'data_augmentation': False, 'gpu_mode': True, 'threads': 1, 'gpus': 1}


ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
2021-05-20 17:49:30.762228: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
usage: tensorboard [-h] [--helpfull] {serve,dev} ...
tensorboard: error: invalid choice: 'tensorboard' (choose from 'serve', 'dev')

===> Epoch[1](1/25): Loss: 0.4875 || Timer: 7.1870 sec.
===> Epoch[1](2/25): Loss: 0.1721 || Timer: 6.6374 sec.
===> Epoch[1](3/25): Loss: 0.0928 || Timer: 6.6232 sec.
===> Epoch[1](4/25): Loss: 0.0571 || Timer: 6.6222 sec.
===> Epoch[1](5/25): Loss: 0.0384 || Timer: 6.6373 sec.
===> Epoch[1](6/25): Loss: 0.0447 || Timer: 6.6352 sec.
===> Epoch[1](7/25): Loss: 0.0627 || Timer: 6.6231 sec.
===> Epoch[1](8/25): Loss: 0.0339 || Timer: 6.6221 sec.
===> Epoch[1](9/25): Loss: 0.0377 || Timer: 6.6159 sec.
===> Epoch[1](10/25): Loss: 0.0314 || Timer: 6.6298 sec.
===> Epoch[1](11/25): Loss: 0.0287 || Timer: 6.6303 sec.
===> Epoch[1](12/25): Loss: 0.0207 || Timer: 6.6290 sec.
===> Epoch[1](13/25): Loss: 0.0200 || Timer: 6.6235 sec.
===> Epoch[1](14/25): Loss: 0.0241 || Timer: 6.6133 sec.
===> Epoch[1](15/25): Loss: 0.0283 || Timer: 6.6307 sec.
===> Epoch[1](16/25): Loss: 0.0201 || Timer: 6.6209 sec.
===> Epoch[1](17/25): Loss: 0.0203 || Timer: 6.6299 sec.
===> Epoch[1](18/25): Loss: 0.0185 || Ti

In [ ]:
# Test

from __future__ import print_function
import argparse

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from functools import reduce
from math import log10

# from scipy.misc import imsave
import scipy.io as sio
import time
import cv2
import easydict
from google.colab import drive
drive.mount('/content/gdrive')


opt = easydict.EasyDict({ 
    "testBatchSize": 1,     # 이미지를 1개씩불러옴 
    "upscale_factor": 2,    # Upscale 정도
    "input_channel": 3,     # WCNN model 입력의 Channel (color = 3, gray = 1)

    "model_type": "WCNN",         # 모델이름
    
    "input_dir":"/content/gdrive/My Drive/졸업논문/data_800개/test",  # test dataset 불러올위치
    "test_dataset": "low_color_test",                             # test에 사용할 dataset 종류
    "output": "/content/gdrive/My Drive/졸업논문/졸업논문/scale=2/insize=3_lr=0.001_patch=64/results/", # 결과영상 저장위치
    "model_type": "WCNN",
    "model": "/content/gdrive/My Drive/졸업논문/졸업논문/scale=2/insize=3_lr=0.001_patch=64/weights/WCNN_epoch_100.pth",
    # test에 사용할 weight 불러올 파일 경로
    
    "pretrained": True,
    "data_augmentation": False,
    "gpu_mode": True,
    "threads": 0, 
    "gpus": 1 # 사용할 gpu 번호
    })

'''
1. GPU 설정 및 parameter print
'''
gpus_list = range(opt.gpus)
cudnn.benchmark = True
print(opt)

cuda = opt.gpu_mode
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

'''
2. Test Data 로드
'''
print('===> Loading datasets')
test_set = get_eval_set(os.path.join(opt.input_dir,opt.test_dataset), opt.upscale_factor)
testing_data_loader = DataLoader(dataset=test_set, num_workers=opt.threads, batch_size=opt.testBatchSize, shuffle=False)

'''
3. Criterion 정의 
  -> MSE loss 사용
'''
criterion = nn.MSELoss() 

'''
4. NN 정의
'''
model = WCNN()

'''
5. pretrained model 및 GPU 사용 여부 설정
'''
if cuda:
    model = model.cuda(gpus_list[0])
    model = torch.nn.DataParallel(model, device_ids=gpus_list)
    criterion = criterion.cuda(gpus_list[0])
if opt.pretrained:
    model_name = os.path.join(opt.model)
    checkpoint = torch.load(model_name)

    model.load_state_dict(checkpoint['model_state_dict'])
    loss = checkpoint['loss']
    epoch = checkpoint['epoch']
    print('Pre-trained SR model is loaded.')
    # pretrained model 불러오기
    
'''
6. 이미지 저장 함수 -> 원본 이미지의 크기 그대로 저장 됨
'''
def save_img(img, img_name):
    save_img = img.squeeze().clamp(0, 1).numpy().transpose(1,2,0)
    # save img
    save_dir=os.path.join(opt.output,opt.test_dataset)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    save_fn = save_dir +'/'+ img_name
    cv2.imwrite(save_fn, cv2.cvtColor(save_img*255, cv2.COLOR_BGR2RGB),  [cv2.IMWRITE_PNG_COMPRESSION, 0])
    
'''
7. 한번의 epoch 에서 수행하는 과정 정의
'''
def eval():
    avg_psnr = 0
    psnr_sq = 0
    model.eval()
    count_image = 0
    # model.eval() 을 호출하여 평가(test) 모드로 변환 
    
    for batch_data in testing_data_loader:
        with torch.no_grad():
            input, bicubic, target, file_name = Variable(batch_data[0]), Variable(batch_data[1]), Variable(batch_data[2]), batch_data[3] 
                                                                
        if cuda:
            input = input.cuda(gpus_list[0])
            bicubic = bicubic.cuda(gpus_list[0])
            target = target.cuda(gpus_list[0])

        prediction = model(bicubic)
        print(prediction.shape, target.shape)

        if prediction.shape != target.shape:
          H = prediction.shape[2]
          W = prediction.shape[3]
          target = target[:, :, 0:H, 0:W]
        # prediction 과 target 의 shape 이 맞지 않을때 맞춰줌
        
        if prediction.shape == target.shape:
          mse = criterion(prediction, target)
          psnr = 10 * log10(1 / mse.item())
          avg_psnr += psnr
          psnr_sq += psnr*psnr
          count_image += 1
        # Image.BICUBIC 에서 이미지의 H나 W가 3의 배수가 아니면 나머지를 버려버리기 때문에 prediction 과 target 의 shape 이 맞지 않는 경우가 생김
        

        save_img(prediction.cpu().data, 'SR_'+file_name[0])
        save_img(bicubic.cpu().data, 'blur_'+file_name[0])
        # save_img(target.cpu().data, file_name[0]+'/original')
       
    average = avg_psnr/count_image
    variance = psnr_sq/count_image-average*average
        
        # psnr 의 분산
    print('Image Amount: %d' %(count_image))
    print("===> epoch number : %d" % (epoch)) 
    print("===> Processing Done, Average PSNR : %.4f" % (average))
    print("===>PSNR Variance : %.4f" % (variance))
    
'''
우리는 훈련시에 128x128 이미지를 SR 하도록 훈련 했지만,
test 에서 입력 이미지의 크기는 상관이 없다
왜냐하면, 우리는 훈련하여 'filter' 를 얻었고,
입력의 크기가 128x128 가 아니더라도 훈련된 filter 를 stride 하면서 Conv. 연산 해 주면 되는 것이기 때문이다.
'''

##Eval Start!!!!
if __name__ == '__main__':
    eval()
    
print('finish')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
{'testBatchSize': 1, 'upscale_factor': 2, 'input_channel': 3, 'model_type': 'WCNN', 'input_dir': '/content/gdrive/My Drive/졸업논문/data_800개/test', 'test_dataset': 'low_color_test', 'output': '/content/gdrive/My Drive/졸업논문/졸업논문/scale=2/results_scale=2/', 'model': '/content/gdrive/My Drive/졸업논문/졸업논문/scale=2/weights_scale=2/WCNN_epoch_100.pth', 'pretrained': True, 'data_augmentation': False, 'gpu_mode': True, 'threads': 0, 'gpus': 1}
===> Loading datasets
Pre-trained SR model is loaded.
torch.Size([1, 3, 436, 700]) torch.Size([1, 3, 437, 700])
torch.Size([1, 3, 534, 800]) torch.Size([1, 3, 534, 800])
torch.Size([1, 3, 532, 800]) torch.Size([1, 3, 532, 800])
Image Amount: 2
===> epoch number : 100
===> Processing Done, Average PSNR : 23.7153
===>PSNR Variance : 0.1042
finish


In [ ]:
# Test - blur 되고 down scale 된 이미지가 아닌 원본 이미지를 입력으로 넣음

from __future__ import print_function
import argparse

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
from functools import reduce
from math import log10

# from scipy.misc import imsave
import scipy.io as sio
import time
import cv2
import easydict
from google.colab import drive
drive.mount('/content/gdrive')

opt = easydict.EasyDict({ 
    "testBatchSize": 1,     # 이미지를 1개씩불러옴 
    "upscale_factor": 3,    # Upscale 정도
    "input_channel": 3,     # ESPCN model 입력의 Channel (color = 3, gray = 1)

    "model_type": "ESPCN",         # 모델이름
    
    "input_dir":"/content/gdrive/My Drive/졸업논문/data_800개/test",  # test dataset 불러올위치
    "test_dataset": "Color_test",                             # test에 사용할 dataset 종류
    "output": "/content/gdrive/My Drive/졸업논문/ESPCN/results/", # 결과영상 저장위치
    "model_type": "ESPCN",
    "model": "/content/gdrive/My Drive/졸업논문/ESPCN/weights/ESPCN_epoch_200.pth",
    # test에 사용할 weight 불러올 파일 경로
    
    "pretrained": True,
    "data_augmentation": False,
    "gpu_mode": True,
    "threads": 0, 
    "gpus": 1 # 사용할 gpu 번호
    })

'''
1. GPU 설정 및 parameter print
'''
gpus_list = range(opt.gpus)
cudnn.benchmark = True
print(opt)

cuda = opt.gpu_mode
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

'''
2. Test Data 로드
'''
print('===> Loading datasets')
test_set = get_eval_set(os.path.join(opt.input_dir,opt.test_dataset), opt.upscale_factor)
testing_data_loader = DataLoader(dataset=test_set, num_workers=opt.threads, batch_size=opt.testBatchSize, shuffle=False)

'''
3. Criterion 정의 
  -> MSE loss 사용
'''
criterion = nn.MSELoss() 

'''
4. NN 정의
'''
model = model = ESPCN(input_channel=3, base_channel=64, upscale_factor=3, activation='tanh', norm='batch')

'''
5. pretrained model 및 GPU 사용 여부 설정
'''
if cuda:
    model = model.cuda(gpus_list[0])
    model = torch.nn.DataParallel(model, device_ids=gpus_list)
    criterion = criterion.cuda(gpus_list[0])
if opt.pretrained:
    model_name = os.path.join(opt.model)
    checkpoint = torch.load(model_name)

    model.load_state_dict(checkpoint['model_state_dict'])
    loss = checkpoint['loss']
    opt.start_epoch = checkpoint['epoch']
    print('Pre-trained SR model is loaded.')
    # pretrained model 불러오기
    
'''
6. 이미지 저장 함수 -> 원본 이미지의 크기 그대로 저장 됨
'''
def save_img(img, img_name):
    save_img = img.squeeze().clamp(0, 1).numpy().transpose(1,2,0)
    # save img
    save_dir=os.path.join(opt.output,opt.test_dataset)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    save_fn = save_dir +'/'+ img_name
    cv2.imwrite(save_fn, cv2.cvtColor(save_img*255, cv2.COLOR_BGR2RGB),  [cv2.IMWRITE_PNG_COMPRESSION, 0])
    
'''
7. 한번의 epoch 에서 수행하는 과정 정의
'''
def eval():
    avg_psnr = 0
    psnr_sq = 0
    model.eval()
    # model.eval() 을 호출하여 평가(test) 모드로 변환 
    
    for batch_data in testing_data_loader:
        with torch.no_grad():
            input, blur_img, target, file_name = Variable(batch_data[0]), Variable(batch_data[1]), Variable(batch_data[2]), batch_data[3] 
                                                                
        if cuda:
            input = input.cuda(gpus_list[0])
            blur_img = blur_img.cuda(gpus_list[0])
            target = target.cuda(gpus_list[0])

        SR_img = model(target)
        # target = 원본 이미지 -> 그러므로 원본 이미지가 그대로 들어가서 3배 scale 됨
        print(SR_img.shape, target.shape)
      
      
        save_img(SR_img.cpu().data, 'Origin_SR_'+file_name[0])
    
'''
우리는 훈련시에 128x128 이미지를 SR 하도록 훈련 했지만,
test 에서 입력 이미지의 크기는 상관이 없다
왜냐하면, 우리는 훈련하여 'filter' 를 얻었고,
입력의 크기가 128x128 가 아니더라도 훈련된 filter 를 stride 하면서 Conv. 연산 해 주면 되는 것이기 때문이다.
'''

##Eval Start!!!!
if __name__ == '__main__':
    eval()
    
print('finish')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
{'testBatchSize': 1, 'upscale_factor': 3, 'input_channel': 3, 'model_type': 'ESPCN', 'input_dir': '/content/gdrive/My Drive/졸업논문/data_800개/test', 'test_dataset': 'Color_test', 'output': '/content/gdrive/My Drive/졸업논문/ESPCN/results/', 'model': '/content/gdrive/My Drive/졸업논문/ESPCN/weights/ESPCN_epoch_200.pth', 'pretrained': True, 'data_augmentation': False, 'gpu_mode': True, 'threads': 0, 'gpus': 1}
===> Loading datasets
Pre-trained SR model is loaded.
torch.Size([1, 3, 6120, 4068]) torch.Size([1, 3, 2040, 1356])
torch.Size([1, 3, 4932, 6120]) torch.Size([1, 3, 1644, 2040])
torch.Size([1, 3, 4068, 6120]) torch.Size([1, 3, 1356, 2040])
torch.Size([1, 3, 4068, 6120]) torch.Size([1, 3, 1356, 2040])
finish
